# Data Fitting COVID19
Exercise for the second lab session.

### Running this notebook on Colab
Run the following snippet if you are running this notebook in [Colab](https://colab.research.google.com/).

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt 
        except:
            pass

## Discrete Time Model
The **SIR model** considers a population of $N$ individuals partitioned into three groups: (i) Susceptible, (ii) Infectious, and (iii) Recovered. The number of individuals at time $t$ is each group is denoted by $S(t)$, $I(t)$, and $R(t)$, respectively. Since the model do not consider births and deaths, we have at any time of outbreak:
$$
    N = S(t) + I(t) + R(t).
$$

The previous model can be discretized thus obtaining the following **discrete time model**:
$$
S(t+1) = S(t) - \frac{\beta}{N} I(t) S(t), \;\;\;\;\;\;\;\;\;\;
$$
$$
I(t+1) = I(t) + \frac{\beta}{N} S(t) I(t) - \gamma I(t), 
$$
$$
R(t+1) = R(t) + \gamma I(t) \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;
$$

Intuitively, we have that proportion of individuals moving at unit of time $t+1$ from the susceptible group $S$ to the infected groups $I$ is equal to $\frac{\beta}{N} I(t) S(t)$. The proportion of individuals moving at unit of time $t+1$ from the infected group $I$ to the recovered groups $R$ is equal to $\gamma I(t)$.

The dynamic of the system is completely described by the two parameters $\beta$ and $\gamma$, and the initial conditions $S(0)$, $I(0)$, and $R(0)$.

> Note: The goal of this post is to present a data fitting approach to estimate the two parameters $\gamma$ and $\beta$, given the data of an outbreak which is not yet concluded. That is, we have reasonable good estimate of $I(t)$ and $R(t)$ for all $t \leq \bar t$, but we have to estimate those values for $t > \bar t$.

### Your Model
You can write your model below.

In [ ]:
import matplotlib.pyplot as plt


def MovingAvg(Ls, w=3):
    """ Moving average of input data, with time window of w days """
    Ws = [0 for i in range(w-1)] + Ls
    return [sum(Ws[i:i+w])/w for i in range(len(Ws)-w+1)]

# Data for Lombardia+Veneto+Emilia Romagna
x = [1, 5, 20, 62, 155, 216, 299, 364, 554, 766, 954, 1425, 1672,
        2021, 2358, 2815, 3278, 4184, 5092, 6470, 6627, 8291, 9951,
        11196, 13183, 14773, 16223, 17987, 19134, 21613, 24186, 27245,
        28919, 31116, 32930, 34592, 37179, 
        39904, 41386, 43178, 43336, 43927]

Ls = MovingAvg(x, 7)

# the barplot of the data
plt.title('Italy - From 24/02 to 26/03')
plt.ylabel('Covid19 Infected')
plt.xlabel('Days')
plt.grid(True)

plt.bar([i for i in range(len(Ls))], Ls, facecolor='green', alpha=0.75)

plt.show()

COMPLETE WITH YOUR CODE BELOW.

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory
from pyomo.environ import maximize, minimize, Binary, RangeSet, ConstraintList

# WRITE YOUR MODEL